In [103]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector
from mysql.connector import Error
import numpy as np

In [74]:
Customer_df = pd.read_csv(r"C:\Users\Sruthika\OneDrive\Desktop\Dataspark project\Customers.csv", encoding='latin-1')
Product_df = pd.read_csv(r"C:\Users\Sruthika\OneDrive\Desktop\Dataspark project\Products.csv", encoding='latin-1')
Sales_df = pd.read_csv(r"C:\Users\Sruthika\OneDrive\Desktop\Dataspark project\Sales.csv", encoding='latin-1')
Stores_df = pd.read_csv(r"C:\Users\Sruthika\OneDrive\Desktop\Dataspark project\Stores.csv", encoding='latin-1')
Exchange_rate_df = pd.read_csv(r"C:\Users\Sruthika\OneDrive\Desktop\Dataspark project\Exchange_Rates.csv", encoding='latin-1')


In [75]:
Customer_df

,CustomerKey,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,7/3/1939
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,9/27/1979
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,5/26/1947
3,786,Male,Jai Poltpalingada,MIDDLE RIVER,SA,South Australia,5223,Australia,Australia,9/17/1957
4,1042,Male,Aidan Pankhurst,TAWONGA SOUTH,VIC,Victoria,3698,Australia,Australia,11/19/1965
...,...,...,...,...,...,...,...,...,...,...
15261,2099600,Female,Denisa Duková,Houston,TX,Texas,77017,United States,North America,3/25/1936
15262,2099618,Male,Justin Solórzano,Mclean,VA,Virginia,22101,United States,North America,2/16/1992
15263,2099758,Male,Svend Petrussen,Wilmington,NC,North Carolina,28405,United States,North America,11/9/1937
15264,2099862,Female,Lorenza Rush,Riverside,CA,California,92501,United States,North America,10/12/1937


In [76]:
Customer_df.isnull().sum()

CustomerKey     0
Gender          0
Name            0
City            0
State Code     10
State           0
Zip Code        0
Country         0
Continent       0
Birthday        0
dtype: int64

In [77]:
Customer_df.dtypes


CustomerKey     int64
Gender         object
Name           object
City           object
State Code     object
State          object
Zip Code       object
Country        object
Continent      object
Birthday       object
dtype: object

In [78]:
def convert_date(date_str):
    try:
        return pd.to_datetime(date_str, format='%m/%d/%Y', errors='coerce').strftime('%m/%d/%Y')
    except ValueError:
        try:
            return pd.to_datetime(date_str, format='%d-%m-%Y', errors='coerce').strftime('%m/%d/%Y')
        except ValueError:
            return date_str 
Customer_df['Birthday'] = Customer_df['Birthday'].apply(convert_date)

In [79]:
Customer_df.fillna({'State Code': 'NAN'}, inplace=True)



In [80]:
Product_df

,ProductKey,Product Name,Brand,Color,Unit Cost USD,Unit Price USD,SubcategoryKey,Subcategory,CategoryKey,Category
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,$6.62,$12.99,101,MP4&MP3,1,Audio
1,2,Contoso 512MB MP3 Player E51 Blue,Contoso,Blue,$6.62,$12.99,101,MP4&MP3,1,Audio
2,3,Contoso 1G MP3 Player E100 White,Contoso,White,$7.40,$14.52,101,MP4&MP3,1,Audio
3,4,Contoso 2G MP3 Player E200 Silver,Contoso,Silver,$11.00,$21.57,101,MP4&MP3,1,Audio
4,5,Contoso 2G MP3 Player E200 Red,Contoso,Red,$11.00,$21.57,101,MP4&MP3,1,Audio
...,...,...,...,...,...,...,...,...,...,...
2512,2513,Contoso Bluetooth Active Headphones L15 Red,Contoso,Red,$43.07,$129.99,505,Cell phones Accessories,5,Cell phones
2513,2514,Contoso Bluetooth Active Headphones L15 White,Contoso,White,$43.07,$129.99,505,Cell phones Accessories,5,Cell phones
2514,2515,Contoso In-Line Coupler E180 White,Contoso,White,$1.71,$3.35,505,Cell phones Accessories,5,Cell phones
2515,2516,Contoso In-Line Coupler E180 Black,Contoso,Black,$1.71,$3.35,505,Cell phones Accessories,5,Cell phones


In [81]:
Product_df.isnull().sum()


ProductKey        0
Product Name      0
Brand             0
Color             0
Unit Cost USD     0
Unit Price USD    0
SubcategoryKey    0
Subcategory       0
CategoryKey       0
Category          0
dtype: int64

In [82]:
Product_df.dtypes


ProductKey         int64
Product Name      object
Brand             object
Color             object
Unit Cost USD     object
Unit Price USD    object
SubcategoryKey     int64
Subcategory       object
CategoryKey        int64
Category          object
dtype: object

In [83]:
Product_df['Unit Cost USD'] = Product_df['Unit Cost USD'].replace('[\$,]', '', regex=True).astype(float)
Product_df['Unit Price USD'] = Product_df['Unit Price USD'].replace('[\$,]', '', regex=True).astype(float)

In [98]:
Sales_df

,Order Number,Line Item,Order Date,Delivery Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code,Delivery_Days
0,366000,1,2016-01-01,2016-01-05,265598,10,1304,1,CAD,NaN
1,366001,1,2016-01-01,2016-01-13,1269051,0,1048,2,USD,12.0
2,366001,2,2016-01-01,2016-01-13,1269051,0,2007,1,USD,12.0
3,366002,1,2016-01-01,2016-01-12,266019,0,1106,7,CAD,11.0
4,366002,2,2016-01-01,2016-01-12,266019,0,373,1,CAD,11.0
...,...,...,...,...,...,...,...,...,...,...
62879,2243030,1,2021-02-20,2021-02-24,1216913,43,632,3,USD,NaN
62880,2243031,1,2021-02-20,2021-02-24,511229,0,98,4,EUR,4.0
62881,2243032,1,2021-02-20,2021-02-23,331277,0,1613,2,CAD,3.0
62882,2243032,2,2021-02-20,2021-02-23,331277,0,1717,2,CAD,3.0


In [85]:
Sales_df.isnull().sum()


Order Number         0
Line Item            0
Order Date           0
Delivery Date    49719
CustomerKey          0
StoreKey             0
ProductKey           0
Quantity             0
Currency Code        0
dtype: int64

In [86]:
# Convert order_date and delivery_date to datetime
Sales_df['Order Date'] = pd.to_datetime(Sales_df['Order Date'])
Sales_df['Delivery Date'] = pd.to_datetime(Sales_df['Delivery Date'])

# Step 1: Calculate the delivery time difference for available delivery dates
Sales_df['Delivery_Days'] = (Sales_df['Delivery Date'] - Sales_df['Order Date']).dt.days

# Step 2: Calculate the median delivery days from available data
median_delivery_days = Sales_df['Delivery_Days'].median()

# Step 3: Fill missing delivery_date using the median delivery time
Sales_df['Delivery Date'] = Sales_df['Delivery Date'].fillna(Sales_df['Order Date'] + pd.to_timedelta(median_delivery_days, unit='D'))


In [87]:
Stores_df

,StoreKey,Country,State,Square Meters,Open Date
0,1,Australia,Australian Capital Territory,595.0,1/1/2008
1,2,Australia,Northern Territory,665.0,1/12/2008
2,3,Australia,South Australia,2000.0,1/7/2012
3,4,Australia,Tasmania,2000.0,1/1/2010
4,5,Australia,Victoria,2000.0,12/9/2015
...,...,...,...,...,...
62,63,United States,Utah,2000.0,3/6/2008
63,64,United States,Washington DC,1330.0,1/1/2010
64,65,United States,West Virginia,1785.0,1/1/2012
65,66,United States,Wyoming,840.0,1/1/2014


In [88]:
Stores_df.isnull().sum()


StoreKey         0
Country          0
State            0
Square Meters    1
Open Date        0
dtype: int64

In [89]:
Stores_df.dtypes


StoreKey           int64
Country           object
State             object
Square Meters    float64
Open Date         object
dtype: object

In [90]:
Stores_df.fillna(0,inplace=True)


In [91]:
Stores_df['Country']=Stores_df['Country'].astype(str)
Stores_df['State']=Stores_df['State'].astype(str)
Stores_df['Open Date']=pd.to_datetime(Stores_df['Open Date'],format='%m/%d/%Y')

In [92]:
Exchange_rate_df

,Date,Currency,Exchange
0,1/1/2015,USD,1.0000
1,1/1/2015,CAD,1.1583
2,1/1/2015,AUD,1.2214
3,1/1/2015,EUR,0.8237
4,1/1/2015,GBP,0.6415
...,...,...,...
11210,2/20/2021,USD,1.0000
11211,2/20/2021,CAD,1.2610
11212,2/20/2021,AUD,1.2723
11213,2/20/2021,EUR,0.8238


In [93]:
Exchange_rate_df.dtypes


Date         object
Currency     object
Exchange    float64
dtype: object

In [94]:
Exchange_rate_df['Date'] = pd.to_datetime(Exchange_rate_df['Date'], format='%m/%d/%Y', errors='coerce')


In [101]:
Cleaned_data=pd.merge(Customer_df, Sales_df, on='CustomerKey',how='inner')


In [104]:
Cleaned_data=pd.merge(Cleaned_data,Product_df,on='ProductKey',how='inner')


In [105]:
Cleaned_data=pd.merge(Cleaned_data,Stores_df,on=['StoreKey'],how='inner')



In [106]:
Cleaned_data

,CustomerKey,Gender,Name,City,State Code,State_x,Zip Code,Country_x,Continent,Birthday,...,Unit Cost USD,Unit Price USD,SubcategoryKey,Subcategory,CategoryKey,Category,Country_y,State_y,Square Meters,Open Date
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,07/03/1939,...,98.07,296.00,104,Recording Pen,1,Audio,Online,Online,0.0,2010-01-01
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,09/27/1979,...,160.49,349.00,303,Desktops,3,Computers,Australia,Western Australia,2000.0,2010-01-01
2,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,09/27/1979,...,72.66,158.00,306,"Printers, Scanners & Fax",3,Computers,Australia,Western Australia,2000.0,2010-01-01
3,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,09/27/1979,...,214.03,646.00,402,Digital SLR Cameras,4,Cameras and camcorders,Australia,Western Australia,2000.0,2010-01-01
4,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,09/27/1979,...,137.63,269.95,303,Desktops,3,Computers,Australia,Western Australia,2000.0,2010-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62879,2099937,Male,Zygmunt Kaminski,Bloomfield Township,MI,Michigan,48302,United States,North America,08/18/1965,...,188.13,369.00,303,Desktops,3,Computers,United States,New Hampshire,1260.0,2015-01-01
62880,2099937,Male,Zygmunt Kaminski,Bloomfield Township,MI,Michigan,48302,United States,North America,08/18/1965,...,186.60,366.00,301,Laptops,3,Computers,United States,New Hampshire,1260.0,2015-01-01
62881,2099937,Male,Zygmunt Kaminski,Bloomfield Township,MI,Michigan,48302,United States,North America,08/18/1965,...,395.48,860.00,405,Camcorders,4,Cameras and camcorders,United States,New Hampshire,1260.0,2015-01-01
62882,2099937,Male,Zygmunt Kaminski,Bloomfield Township,MI,Michigan,48302,United States,North America,08/18/1965,...,275.46,599.00,303,Desktops,3,Computers,United States,Montana,1260.0,2012-06-06
